# Modeling

 - 방법 1. `nn.Sequential`
 - 방법 2. Sub-class of `nn.Module`

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

### `nn.Sequential`


In [2]:
nn.Conv2d(
    in_channels=3, # input 채널의 수를 지정해주어야함
    out_channels=32,
    kernel_size=3,
    stride=1,
    padding=1,
    bias=False
)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [3]:
nn.Linear(
    in_features=784,
    out_features=500,
    bias=False
)

Linear(in_features=784, out_features=500, bias=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model = nn.Sequential(
    nn.Linear(784, 15),
    nn.Sigmoid(),
    nn.Linear(15, 10),
    nn.Sigmoid(),
).to(device)

In [6]:
!pip install torchsummary

In [7]:
import torchsummary

In [8]:
# model의 정보를 좀 더 자세하고 이쁘게 출력 가능

torchsummary.summary(model, (784, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 15]          11,775
           Sigmoid-2                   [-1, 15]               0
            Linear-3                   [-1, 10]             160
           Sigmoid-4                   [-1, 10]               0
Total params: 11,935
Trainable params: 11,935
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


### `nn.module` sub class

- `__init__()` 에서 Layers를 초기화 함.
- `forward` 함수를 구현

In [9]:
# (1, 28, 28) input size 가정

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4900, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x)) #(28, 28)
        x = F.max_pool2d(x, 2, 2) #(14, 14)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2) #(7, 7)

        x = x.view(-1, 4900)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

In [10]:
model = Net()
torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 28, 28]             200
            Conv2d-2           [-1, 50, 14, 14]           9,050
            Linear-3                  [-1, 500]       2,450,500
            Linear-4                   [-1, 10]           5,010
Total params: 2,464,760
Trainable params: 2,464,760
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.20
Params size (MB): 9.40
Estimated Total Size (MB): 9.60
----------------------------------------------------------------


### 간단한 ResNet 구현

In [11]:
# ResidualBlock

class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(ResidualBlock, self).__init__()

        self.in_channel, self.out_channel = in_channel, out_channel

        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channel, out_channel, kernel_size=1, padding=0)

        if in_channel != out_channel:
          # in_channel과 out_channel이 맞지 않을 시
          # 더해주기 전 한 번 더 channel을 맞춰주는 작업 실시
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out += self.shortcut(x)
        return out

In [12]:
class ResNet(nn.Module):
    def __init__(self, color='gray'):
        super(ResNet, self).__init__()
        # 컬러에 따라 맨 첫번째 레이어을 다르게 작업
        if color == "gray":
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        elif color == "rgb":
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 64)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1)) # averagepool
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10) # output에 맞게

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [13]:
model = ResNet()
torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 14, 14]           2,112
            Conv2d-3           [-1, 64, 14, 14]          36,928
            Conv2d-4           [-1, 64, 14, 14]           4,160
            Conv2d-5           [-1, 64, 14, 14]           2,112
     ResidualBlock-6           [-1, 64, 14, 14]               0
            Conv2d-7           [-1, 64, 14, 14]           4,160
            Conv2d-8           [-1, 64, 14, 14]          36,928
            Conv2d-9           [-1, 64, 14, 14]           4,160
    ResidualBlock-10           [-1, 64, 14, 14]               0
AdaptiveAvgPool2d-11             [-1, 64, 1, 1]               0
           Linear-12                   [-1, 64]           4,160
           Linear-13                   [-1, 10]             650
Total params: 95,690
Trainable params: 